<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №1**

Необходимо провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Загрузка и предобработка данных:

*   из исходного DataFrame выбираются только интересующие столбцы и удаляются строки, в которых есть хотя бы одно пропущенное значение в выбранных столбцах с помощью dropna().
*   преобразование категориальных данных в столбце 'city' в числовой формат.
* разделение данных на признаки и целевую переменную.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np
import matplotlib.pyplot as plt


data = pd.read_csv('/content/drive/MyDrive/train 3.csv', on_bad_lines='skip')
print(data.head())

data = data[['log_price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'property_type']].dropna()

# Разделение цены на категории по квантилям
data['price_category'] = pd.qcut(data['log_price'], q=4, labels=False)  # разделение на 4 категории
# Подготовка признаков и целевой переменной
X = data[['accommodates', 'bathrooms', 'bedrooms', 'beds']]
y = data['price_category']

         id  log_price property_type        room_type  \
0   6901257   5.010635     Apartment  Entire home/apt   
1   6304928   5.129899     Apartment  Entire home/apt   
2   7919400   4.976734     Apartment  Entire home/apt   
3  13418779   6.620073         House  Entire home/apt   
4   3808709   4.744932     Apartment  Entire home/apt   

                                           amenities  accommodates  bathrooms  \
0  {"Wireless Internet","Air conditioning",Kitche...             3        1.0   
1  {"Wireless Internet","Air conditioning",Kitche...             7        1.0   
2  {TV,"Cable TV","Wireless Internet","Air condit...             5        1.0   
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...             4        1.0   
4  {TV,Internet,"Wireless Internet","Air conditio...             2        1.0   

   bed_type cancellation_policy  cleaning_fee  ...   latitude   longitude  \
0  Real Bed              strict          True  ...  40.696524  -73.991617   
1  Real Bed     

Использование функции train_test_split из библиотеки sklearn.model_selection для разделения данных на обучающую (X_train, y_train) и тестовую (X_test, y_test) выборки. Аргумент test_size=0.2 указывает, что 20% данных должны быть использованы для тестовой выборки. Параметр random_state=42 обеспечивает воспроизводимость результатов разделения.

Нормализация данных с помощью StandardScaler()

In [14]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание пайплайна, включающего нормализацию данных и классификатор
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])


Лучшие параметры: {'knn__metric': 'manhattan', 'knn__n_neighbors': 17}
Лучшая модель: Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=17))])


In [ ]:
# Параметры для GridSearchCV
params = {
    'knn__n_neighbors': list(range(1, 20)),
    'knn__metric': ['euclidean', 'minkowski', 'manhattan']
}


grid_search = GridSearchCV(pipe, param_grid=params, cv=5)  # cv - число фолдов кросс-валидации


grid_search.fit(X_train, y_train)


print('Лучшие параметры:', grid_search.best_params_)

Процесс обучения модели классификации на основе алгоритма K-ближайших соседей (KNN), а также её последующая оценка

In [15]:
# Обучение модели
optimal_k = grid.best_params_['knn__n_neighbors']
pipe.set_params(knn__n_neighbors=optimal_k)
pipe.fit(X_train, y_train)


y_pred = pipe.predict(X_test)


print(classification_report(y_test, y_pred))

print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())

              precision    recall  f1-score   support

           0       0.53      0.40      0.46      2883
           1       0.33      0.65      0.43      2875
           2       0.34      0.16      0.22      2856
           3       0.71      0.54      0.61      2812

    accuracy                           0.44     11426
   macro avg       0.48      0.44      0.43     11426
weighted avg       0.48      0.44      0.43     11426

Границы категорий цен: [(4.7, 5.165], (4.304, 4.7], (-0.001, 4.304], (5.165, 7.6]]
Categories (4, interval[float64, right]): [(-0.001, 4.304] < (4.304, 4.7] < (4.7, 5.165] <
                                           (5.165, 7.6]]


* Precision (точность): Показывает долю правильных предсказаний среди всех предсказаний данного класса. Например, для класса 0 точность составляет 0.53, что означает, что 53% предсказанных как класс 0 действительно принадлежат классу 0.
* Recall (полнота): Показывает, какая доля реальных объектов данного класса была правильно классифицирована. Для класса 0 полнота составляет  0.40, что указывает на то, что модель смогла правильно определить  40% всех реальных объектов класса 0.
* F1-score: Гармоническое среднее между точностью и полнотой. Высокий показатель F1-score указывает на то, что модель действует сбалансированно в отношении точности и полноты. Для класса 0 F1-score составляет 0.46, что указывает на среднюю эффективность модели в этом классе.
* Support: Число реальных объектов каждого класса в тестовой выборке. Например, класс 0 содержит 2883 объекта.
*   Accuracy (точность всей модели): 0.44, или 44%, что говорит о том, что в 44% случаев модель правильно предсказала класс объекта.
*   Macro avg: Среднее значение каждой метрики по всем классам, не учитывающее число объектов в каждом классе. Показатели довольно низкие (особенно F1-score), что может свидетельствовать о неравномерной производительности модели по различным классам.
* Weighted avg: Взвешенное среднее каждой метрики, где вес каждого класса пропорционален числу объектов в нём. Эти значения обычно более репрезентативны для несбалансированных данных.
